In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
from datetime import datetime
from time import time
import glob
import itertools
from scipy import spatial
import cv2
from multiprocessing.pool import ThreadPool as Pool

ModuleNotFoundError: No module named 'multiprocessing.pool'; 'multiprocessing' is not a package

In [2]:
# new_img_path='C:/Users/ThamotharanC/OneDrive - Softcrylic LLC/Pictures/Camera Roll/image/1.png'
new_img_path = "C:/Users/ThamotharanC/OneDrive - Softcrylic LLC/Desktop/Django/facerec/facerecapp/static/temp_img/image.png"
image=cv2.imread(new_img_path)

print("crop after click spacebar")

roi=cv2.selectROI(image)
print(roi)

im_cropped = image[int(roi[1]):int(roi[1]+roi[3]),int(roi[0]):int(roi[0]+roi[2])]

# cv2.imshow("Cropped Image", im_cropped)
cv2.waitKey(1)
cv2.destroyAllWindows()

crop after click spacebar
(104, 358, 125, 30)


In [4]:
def templatenew(i): 
    output_path='C:/Users/ThamotharanC/OneDrive - Softcrylic LLC/Pictures/Camera Roll/thamu'
    pa=output_path
#     newcv=cv2.imread(new_img_path)      #small
    newcv=im_cropped
    var=""
    if(i.endswith('.bmp') or i.endswith('.jpg') or i.endswith('.png') or i.endswith('.jpeg')):
            imfo=i
            serread=cv2.imread(i)  #big 
            sercv=serread
#             sercv=serread[300:500, :]
#             sercv=sercv[int(roi[1]):int(roi[1]+roi[3]),int(roi[0]):int(roi[0]+roi[2])]
            nh,nw,ns=newcv.shape
            sh,sw,ss=sercv.shape
            if(sh>=nh and sw>=nw and ss>=ns):
                res=cv2.matchTemplate(sercv,newcv,cv2.TM_CCOEFF_NORMED)
                th=0.80
                loc=np.where(res>=th)
                if(len(loc[0])>0 and len(loc[1])):
                    var="yes"
                    savepath=pa+"/"+"pool test/"+"sample8"+"/"+str(time())+".png"
                    for pt in zip(*loc[::-1]):
                        cv2.rectangle(sercv,pt,(pt[0]+nw,pt[1]+nh),(0,255,255),2)
                    cv2.imwrite(savepath,sercv)
                    cv2.destroyAllWindows()
                else:
                    var="no"
            else:
                var="no"
            return (i,var)
           
            

In [5]:
pool_selected=[]
pool_path=[]
# search_path='C:/Users/ThamotharanC/OneDrive - Softcrylic LLC/Pictures/Camera Roll/image'
search_path = "C:/Users/ThamotharanC/OneDrive - Softcrylic LLC/Desktop/Django/facerec/facerecapp/static/img"
path_list=[search_path+"/"+i for i in os.listdir(search_path)]

In [6]:
# from multiprocessing.pool import ThreadPool as Pool
st=time()
print(datetime.now())

pool = Pool(10)
if __name__=="__main__":
    for path,var in pool.map(templatenew,path_list):
        pool_path.append(path)
        pool_selected.append(var)
        
et=time()
print(et-st)
print(datetime.now())

2022-11-13 23:53:07.464321
158.65184783935547
2022-11-13 23:55:46.116169


# Detect rectangular and square box 

In [ ]:
import cv2
import numpy as np


file='C:/Users/ThamotharanC/OneDrive - Softcrylic LLC/Pictures/Camera Roll/image/t2.png'

# Read image
img = cv2.imread(file)

# threshold red
# lower = np.array([0, 0, 0])
# upper = np.array([40, 40, 255])

lower = np.array([0,0,150])
upper = np.array([40, 40, 255])
thresh = cv2.inRange(img, lower, upper)

# Change non-red to white
# result = img.copy()
# result[thresh != 255] = (255,255,255)

contours,hierarchy = cv2.findContours(thresh, 1, 2)

for cnt in contours:
    x1,y1 = cnt[0][0]
    approx = cv2.approxPolyDP(cnt, 0.01*cv2.arcLength(cnt, True), True)

    if len(approx) == 4:
        x, y, w, h = cv2.boundingRect(cnt)
#         print(x, y, w, h)
        ratio = float(w)/h
        if(w*h > 100):
            if ratio >= 0.9 and ratio <= 1.1:
                img = cv2.drawContours(img, [cnt], -1, (0,255,255), 1)
                cv2.putText(img, 'Square', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)

                
#                 print([cnt])
            else:
                cv2.putText(img, 'Rectangle', (x1, y1), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 1)
                img = cv2.drawContours(img, [cnt], -1, (0,255,0), 1)
        
#                 print(x1,y1)
#                 print([cnt])
    

# save results
# cv2.imwrite('red_numerals_thresh.jpg', thresh)
# cv2.imwrite('red_numerals_result.jpg', result)

# cv2.imshow('thr',thresh)
cv2.imshow('thresh', img)
# cv2.imshow('result', result)
cv2.waitKey(0)
cv2.destroyAllWindows()